# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [229]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
import statsmodels.api as sm   
import patsy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


import warnings

# Ignorar FutureWarnings específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils.validation")

**Importando dados e organizando informações relevantes:**<br>
**Importing data and organizing relevant information:**


In [230]:
df = pd.read_csv('previsao_de_renda.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [231]:
drop = df[['Unnamed: 0','data_ref','id_cliente']]
df = df.drop(drop, axis = 1)
df

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [232]:
df.isnull().sum()

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [233]:
df = df.dropna()

In [234]:
df = df.drop_duplicates()

In [235]:
df = df.reset_index(drop=True)

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8655 entries, 0 to 8654
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   8655 non-null   object 
 1   posse_de_veiculo       8655 non-null   bool   
 2   posse_de_imovel        8655 non-null   bool   
 3   qtd_filhos             8655 non-null   int64  
 4   tipo_renda             8655 non-null   object 
 5   educacao               8655 non-null   object 
 6   estado_civil           8655 non-null   object 
 7   tipo_residencia        8655 non-null   object 
 8   idade                  8655 non-null   int64  
 9   tempo_emprego          8655 non-null   float64
 10  qt_pessoas_residencia  8655 non-null   float64
 11  renda                  8655 non-null   float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 693.2+ KB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

**1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento):**

In [237]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=62)

In [238]:
modelo = '''renda ~ C(sexo, Treatment(0)) 
            + C(posse_de_veiculo, Treatment(0))
            + C(posse_de_imovel, Treatment(1)) 
            + qtd_filhos + C(tipo_renda, Treatment(0)) 
            + C(educacao, Treatment(2)) 
            + C(estado_civil, Treatment(0)) 
            + C(tipo_residencia, Treatment(1))
            + idade + tempo_emprego + qt_pessoas_residencia'''

In [239]:
y_train, X_train = patsy.dmatrices(modelo, df_train)
y_test, X_test = patsy.dmatrices(modelo, df_test)

<br>**2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?**

- A regressão **Lasso** utiliza uma penalização **L1**, na **configuração para Lasso(L1) = L1_wt=1.0.** No Lasso, a penalização é **capaz de zerar coeficientes**, levando à seleção de variáveis.<br><br>
- A regressão **Ridge** utiliza uma penalização **L2**, na **configuração para Ridge(L2)= L1_wt=0.0.** No Ridge, ela apenas os reduz, **mantendo-os próximos de zero, mas raramente zerando-os.**


In [240]:
# Ridge L1_wt=0.0
r_quad_pred = []
alphas = [0,0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,
                             alpha=alpha)
    y_predito = reg.predict(X_test)
    r_quad = r2_score(y_test, y_predito)
    r_quad_pred.append(r_quad)
    
print('O melhor R² está definido como:')        
pd.DataFrame({'alphas':alphas, 
              '𝑅2':r_quad_pred}).sort_values(by='𝑅2',
                                             ascending=False).head(1)

O melhor R² está definido como:


,alphas,𝑅2
0,0.0,0.20941


<br>**3. Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?**

In [241]:
# LASSO L1_wt=1
r_quad_pred = []
alphas = [0,0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=alpha)
    y_predito = reg.predict(X_test)
    r_quad = r2_score(y_test, y_predito)
    r_quad_pred.append(r_quad)
    
print('O melhor R² está definido como:')        
pd.DataFrame({'alphas':alphas, 
              '𝑅2':r_quad_pred}).sort_values(by='𝑅2',
                                             ascending=False).head(1)

O melhor R² está definido como:


,alphas,𝑅2
4,0.05,0.209458


<br>**4. Rode um modelo stepwise. Avalie o R²
 na base de testes. Qual o melhor resultado**?

In [242]:
df_dummies = pd.get_dummies(df, drop_first=True).astype(int)
X = df_dummies.drop('renda', axis=1)
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=62)

In [243]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.46071e-65
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.37624e-50
#############
['tempo_emprego', 'sexo_M']
Add  idade                          with p-value 0.00215938
#############
['tempo_emprego', 'sexo_M', 'idade']
Add  tipo_renda_Empresário          with p-value 0.0165505
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário']
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário']


resulting features:
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário']


In [244]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=62)

md = sm.OLS(y_train, X_train)
reg = md.fit()
y_pred=reg.predict(X_test)
r_quad=r2_score(y_test, y_pred)

print('R²:', round(r_quad, 2))

R²: 0.21


In [245]:
# Ridge L1_wt=0.0
r_quad_pred = []
alphas = [0,0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,
                             alpha=alpha)
    y_predito = reg.predict(X_test)
    r_quad = r2_score(y_test, y_predito)
    r_quad_pred.append(r_quad)
    
print('O melhor R² está definido como:')        
pd.DataFrame({'alphas':alphas, 
              '𝑅2':r_quad_pred}).sort_values(by='𝑅2',
                                             ascending=False)

O melhor R² está definido como:


,alphas,𝑅2
0,0.000,0.205934
1,0.001,0.205858
2,0.005,0.205537
3,0.010,0.205105
4,0.050,0.200892
5,0.100,0.195002


In [246]:
# LASSO L1_wt=1
r_quad_pred = []
alphas = [0,0.001, 0.005, 0.01, 0.05, 0.1]

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=alpha)
    y_predito = reg.predict(X_test)
    r_quad = r2_score(y_test, y_predito)
    r_quad_pred.append(r_quad)
    
print('O melhor R² está definido como:')        
pd.DataFrame({'alphas':alphas, 
              '𝑅2':r_quad_pred}).sort_values(by='𝑅2',
                                             ascending=False)

O melhor R² está definido como:


,alphas,𝑅2
0,0.000,0.205934
1,0.001,0.205934
2,0.005,0.205934
3,0.010,0.205934
4,0.050,0.205934
5,0.100,0.205934


- O modelo stepwise com regressão LASSO teve o melhor desempenho, alcançando resultados consistentes e equilibrados em todos os diferentes valores de alpha em relação ao R².

<br>**5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?**

- O modelo stepwise com regressão LASSO foi sem dúvidas a melhor opção, podemos verificar que mesmo com número de colunas inferior aos demais modelos treinados inicialmente obteve um valor de R² equilibrado para o modelo.

<br>**6. Partindo dos modelos que você ajustou, tente melhorar o R² na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.**

In [247]:
df_data = pd.get_dummies(df, drop_first=True).astype(int)
df_data = df_data[['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'renda']]

data_train, data_test = train_test_split(df_data, test_size=0.25, random_state=62)

modelo = smf.ols('''np.log(renda) ~ tempo_emprego 
                                    + sexo_M 
                                    + idade 
                                    + tipo_renda_Empresário'''
                 , data_train)

reg = modelo.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=0)

print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     719.7
Date:                Tue, 12 Dec 2023   Prob (F-statistic):               0.00
Time:                        16:05:24   Log-Likelihood:                -7062.6
No. Observations:                6491   AIC:                         1.414e+04
Df Residuals:                    6486   BIC:                         1.418e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 7.29

In [248]:
y_pred = reg.predict(data_test)
r_quad = r2_score(data_test['renda'], np.exp(y_pred))

print('R²- base teste:', round(r_quad, 2))

R²- base teste: 0.26


<br>**7. Ajuste uma árvore de regressão e veja se consegue um R² melhor com ela.**.

In [266]:
y = df_data[['renda']]
X = df_data[['tempo_emprego','sexo_M', 'idade', 'tipo_renda_Empresário']]

In [267]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=62)

reg = DecisionTreeRegressor(random_state=62, max_depth=5, min_samples_leaf=10)
reg.fit(X_train, y_train)
r = reg.score(X_test, y_test)

In [269]:
print('R²- base teste:', round(r,2))

R²- base teste: 0.45
